In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
%matplotlib inline

In [2]:
path = os.path.join('..', 'Dataset', 'segmentation_train.csv')

df = pd.read_csv(path)

In [42]:
df.head(10)

,CLASS,REGION-CENTROID-COL,REGION-CENTROID-ROW,REGION-PIXEL-COUNT,SHORT-LINE-DENSITY-5,SHORT-LINE-DENSITY-2,VEDGE-MEAN,VEDGE-SD,HEDGE-MEAN,HEDGE-SD,INTENSITY-MEAN,RAWRED-MEAN,RAWBLUE-MEAN,RAWGREEN-MEAN,EXRED-MEAN,EXBLUE-MEAN,EXGREEN-MEAN,VALUE-MEAN,SATURATION-MEAN,HUE-MEAN
0,BRICKFACE,140,125,9,0.000000,0.0,0.277778,0.062963,0.666667,0.311111,6.185185,7.333334,7.666666,3.555556,3.444444,4.444445,-7.888889,7.777778,0.545635,-1.121818
1,BRICKFACE,188,133,9,0.000000,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
2,BRICKFACE,105,139,9,0.000000,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
3,BRICKFACE,34,137,9,0.000000,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
4,BRICKFACE,39,111,9,0.000000,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
5,BRICKFACE,16,128,9,0.000000,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811
6,BRICKFACE,26,67,9,0.111111,0.0,1.000000,0.888890,2.444445,3.185185,20.000000,19.555555,25.888890,14.555555,-1.333333,17.666666,-16.333334,25.888890,0.436939,-1.623202
7,BRICKFACE,14,110,9,0.000000,0.0,1.722222,5.351850,2.666667,1.022223,17.925926,18.888890,21.444445,13.444445,2.888889,10.555555,-13.444445,21.444445,0.368848,-1.345096
8,BRICKFACE,11,108,9,0.000000,0.0,1.333333,0.800000,1.388889,0.951852,17.666666,19.000000,21.111110,12.888889,4.000000,10.333333,-14.333333,21.111110,0.388756,-1.302133
9,BRICKFACE,85,101,9,0.000000,0.0,1.333333,1.288888,1.277778,1.218518,21.296297,21.222221,26.777779,15.888889,-0.222222,16.444445,-16.222221,26.777779,0.404792,-1.558599


In [4]:
x = df.drop('CLASS', axis = 1)
y = df['CLASS']

In [5]:
y = LabelEncoder().fit_transform(y)

In [6]:
total_class = np.unique(y).shape[0]

In [7]:
total_data = 0

In [8]:
class_wise_data = dict([(i, []) for i in range(total_class)])

In [9]:
#Initialize Prior Proabilities
class_prior = dict([(i, 1 / total_class) for i in range(total_class)])
class_wise_probs = dict()

In [10]:
def normal_pdf( x, mu, sigma):
        
        '''
        Proabability distribution function for Normal Distribution
        '''
        scale_parm = 1 / (np.sqrt(2 * np.pi) * sigma)
        shift = np.e ** -(((x - mu) ** 2) / (2 * sigma ** 2))
        return scale_parm * shift

In [11]:
def calc_mu_std(x):
    return {'mu': np.mean(x, axis = 0), 'sigma': np.std(x, axis = 0)}

In [12]:
def update_probs(x, y):
    global class_wise_data, class_prior, class_wise_probs, total_data
    class_wise_data[y].append(x)
    total_data += 1
    for i in class_wise_data.keys():
        class_wise_probs[i] = calc_mu_std(class_wise_data[i])
        class_prior[i] = len(class_wise_data[i]) / total_data

In [38]:
def predict(x):
    global total_class, class_prior, class_wise_probs
    class_probs = []
    for i in range(total_class):
        likelihood_prob = normal_pdf(x, class_wise_probs.get(i, {}).get('mu', 0), class_wise_probs.get(i, {}).get('sigma', 0))
        class_probs = likelihood_prob * class_prior.get(i)
    class_probs = np.array(class_probs)
    final_class = np.argmax(class_probs / np.sum(class_probs))
    return final_class

In [39]:
predict(x.iloc[0])

C:\Users\ritik\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [17]:
for i in range(1000):
    index = np.random.choice(np.arange(0, x.shape[0]))
    print ('Predicted Label: ', predict(x.iloc[index]))
    print ('Actual Label:', y[index])
    update_probs(x.iloc[index], y[index])

C:\Users\ritik\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  
C:\Users\ritik\Anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\ritik\Anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\ritik\Anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\ritik\Anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\ritik\Anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rco

Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual L

Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual L

Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual L

Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 5
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 3
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 6
Predicted Label:  0
Actual Label: 0
Predicted Label:  0
Actual Label: 1
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual Label: 2
Predicted Label:  0
Actual Label: 4
Predicted Label:  0
Actual L

C:\Users\ritik\Anaconda3\envs\tf_gpu\lib\site-packages\pandas\core\ops.py:1533: RuntimeWarning: divide by zero encountered in true_divide
  lambda x: op(x, rvalues))
C:\Users\ritik\Anaconda3\envs\tf_gpu\lib\site-packages\pandas\core\ops.py:1533: RuntimeWarning: invalid value encountered in true_divide
  lambda x: op(x, rvalues))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [18]:
class_prior

{0: 0.1489841986455982,
 1: 0.1433408577878104,
 2: 0.15914221218961624,
 3: 0.14446952595936793,
 4: 0.14221218961625282,
 5: 0.1252821670428894,
 6: 0.136568848758465}

In [25]:
class_wise_probs

{0: {'mu': array([ 8.88939394e+01,  1.12121212e+02,  9.00000000e+00,  3.36700333e-03,
          0.00000000e+00,  1.00715490e+00,  8.57547633e-01,  1.27483165e+00,
          7.62541989e-01,  1.23501684e+01,  1.28779460e+01,  1.55311448e+01,
          8.64141421e+00,  1.58333331e+00,  9.54292928e+00, -1.11262624e+01,
          1.58190236e+01,  4.93473190e-01, -1.29076048e+00]),
  'sigma': array([6.87767863e+01, 2.66728299e+01, 0.00000000e+00, 1.90466471e-02,
         0.00000000e+00, 5.66161631e-01, 9.92874220e-01, 7.73899512e-01,
         7.35499604e-01, 7.49549941e+00, 6.73272331e+00, 9.75399152e+00,
         6.07944963e+00, 3.41875330e+00, 7.06093735e+00, 4.28883219e+00,
         9.50223362e+00, 7.37246735e-02, 3.52313073e-01])},
 1: {'mu': array([ 1.62511811e+02,  9.91181102e+01,  9.00000000e+00,  1.04986876e-02,
          1.22484688e-02,  2.95144372e+00,  2.34546595e+00,  2.65223092e+00,
          2.94347604e+00,  4.29912513e+01,  3.84278216e+01,  5.28118984e+01,
          3.77340333

In [22]:
x.shape

(210, 19)

In [32]:
len(class_wise_data[4])

126